### Import library

In [1]:
import re
import string
import numpy as np
import pandas as pd
from collections import Counter

### Data Preprocessing

In [2]:
# Helper Functions

def process_data(file_name):
    with open(file_name, "r") as f:
        data = f.read()
        
    data = data.lower()
    words = re.findall("\w+", data)
    
    return words

def get_count(word_l):
    return Counter(word_l)

def get_probs(word_count_dict):
    probs = {}
    
    m = sum(word_count_dict.values())
    for key in word_count_dict:
        probs[key] = word_count_dict[key]/m
    
    return probs

In [3]:
word_l = process_data("Dataset/shakespeare.txt")

vocab = set(word_l)
word_count_dict = get_count(word_l)
word_probs = get_probs(word_count_dict)

print("Vocab size :", len(vocab))
print("Word Count for the word 'muse' :", word_count_dict.get("muse", 0))
print(f"P('muse') :", word_probs["muse"])

Vocab size : 6116
Word Count for the word 'muse' : 18
P('muse') : 0.000335733204013877


### String Manipulations

In [4]:
# Helper Functions

def delete_letter(word):
    return [word[:idx]+word[idx+1:] for idx in range(len(word))]

def switch_letter(word):
    word_l = list(word)
    
    req_l = []
    for idx in range(len(word)-1):
        word_l[idx], word_l[idx+1] = word_l[idx+1], word_l[idx]
        req_l.append("".join(word_l))
        word_l[idx], word_l[idx+1] = word_l[idx+1], word_l[idx]
        
    return req_l

def replace_letter(word):
    word_l = list(word)
    
    req_l = []
    for idx in range(len(word)):
        for char in string.ascii_lowercase:
            word_l[idx] = char
            req_l.append("".join(word_l))
            word_l[idx] = word[idx]
            
    req_set = set(req_l)
    req_set.remove(word)
    req_l = sorted(list(req_set))
            
    return req_l

def insert_letter(word):
    req_l = []
    for idx in range(len(word)+1):
        for char in string.ascii_lowercase:
            req_l.append(word[:idx]+ char + word[idx:])
            
    return req_l

In [5]:
print("Delete Letter :", len(delete_letter("at")))
print("Switch Letter :", len(switch_letter("at")))
print("Replace Letter :", len(replace_letter("at")))
print("Insert Letter :", len(insert_letter("at")))

Delete Letter : 2
Switch Letter : 1
Replace Letter : 50
Insert Letter : 78


### Combining the edits

In [6]:
def edit_one_letter(word):
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    
    return edit_one_set


def edit_two_letter(word):
    edit_two_set = set()
    
    edit_one = edit_one_letter(word)
    for w in edit_one:
        if w:
            edit_two_set.update(edit_one_letter(w))
            
    return edit_two_set

In [7]:
print("Edit one Letter :", len(edit_one_letter("at")))
print("Edit Two Letter :", len(edit_two_letter("at")))

Edit one Letter : 129
Edit Two Letter : 7154


### Spelling Suggestions


In [8]:
def get_corrections(word, probs, vocab, n=5):
    
    if (not word) or (word in vocab):
        return [word]
    
    suggestions = list(edit_one_letter(word).intersection(vocab) or edit_two_letter(word).intersection(vocab))
    best = sorted([[s, probs[s]] for s in suggestions], key = lambda x : x[1], reverse=True)
    n_best = [word for word, prob in best[:n]]
    
    return n_best

In [9]:
get_corrections("battl", word_probs, vocab)

['battle', 'batt']

### Minimum Edit distance

In [10]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    m = len(source)
    n = len(target)
    
    arr = np.zeros((m+1, n+1), dtype=int)
    
    for row in range(1, m+1):
        arr[row][0] = arr[row-1][0] + del_cost
        
    for col in range(1, n+1):
        arr[0][col] = arr[0][col-1] + ins_cost
    
    for row in range(1, m+1):
        for col in range(1, n+1):
            
            r_cost = rep_cost
            
            if source[row-1] == target[col-1]:
                r_cost = 0
            
            arr[row][col] = min([arr[row-1][col] + del_cost, arr[row][col-1] + ins_cost, arr[row-1][col-1] + r_cost])
            
    min_edit = arr[m][n]
    
    return arr, min_edit

In [11]:
arr, dist = min_edit_distance("stay", "play")

print(pd.DataFrame(arr, index=list("#stay"), columns=list("#play")))

   #  p  l  a  y
#  0  1  2  3  4
s  1  2  3  4  5
t  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


### Playground

In [12]:
misspelled_string = input("Enter a Word : ")
misspelled_string = misspelled_string.lower()

string_l = get_corrections(misspelled_string, word_probs, vocab, n=10)

print("\nSpelling Suggestions with Minimum Edit Distance\n")

for word in string_l:
    print(word, " : ", min_edit_distance(misspelled_string, word, 1, 1, 1)[1])

Enter a Word : Rahul

Spelling Suggestions with Minimum Edit Distance

rail  :  2
rful  :  2
